In [ ]:
!pip install clickhouse_connect
!pip install transformers
!pip install datasets
!pip install sentencepiece

# Connected Clickhouse server

In [2]:
import clickhouse_connect

# Create a ClickHouse client instance
client = clickhouse_connect.get_client(host='l3zocnvemg.europe-west4.gcp.clickhouse.cloud',
                                       port=8443,
                                       username='default',
                                       database='marketing_agency',
                                       password='MDHXEfD4aO0.k')

In [5]:
query = 'SELECT * from Customer_payments limit 5'
sql_query = client.query(query)
sql_query.result_set

[(2938216809221281166, '100000062', 162.0, datetime.date(2022, 1, 1)),
 (5805451071876313119, '100000031', 131.0, datetime.date(2022, 1, 4)),
 (7748364034143247723, '100000059', 159.0, datetime.date(2022, 1, 6)),
 (3392984714632099274, '100000042', 142.0, datetime.date(2022, 1, 8)),
 (10228746872420000542, '100000062', 162.0, datetime.date(2022, 1, 9))]

# Model implementation

In [6]:
import torch
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

train_data = load_dataset('wikisql', split = 'train')
test_data = load_dataset('wikisql', split = 'test')

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

Dataset wikisql downloaded and prepared to /root/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d. Subsequent calls will reuse this data.


In [8]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
def tokenize_data(data):
    input_texts = []
    output_texts = []
    for sample in data:
        input_texts.append(sample['question'])
        output_texts.append(sample['sql']['human_readable'])

    input_data = tokenizer.batch_encode_plus(input_texts, padding='longest', truncation=True, return_tensors='pt')
    output_data = tokenizer.batch_encode_plus(output_texts, padding='longest', truncation=True, return_tensors='pt')
    return input_data, output_data

train_input_data, train_output_data = tokenize_data(train_data)
test_input_data, test_output_data = tokenize_data(test_data)

In [10]:
batch_size = 16
train_loader = DataLoader(list(zip(train_input_data['input_ids'], train_output_data['input_ids'])), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(list(zip(test_input_data['input_ids'], test_output_data['input_ids'])), batch_size=batch_size)

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

In [ ]:
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [15]:
# Test the model
model.eval()
total = 0
correct = 0

In [ ]:
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model.generate(input_ids=inputs)
        predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        targets = tokenizer.batch_decode(labels, skip_special_tokens=True)

        total += len(predictions)
        correct += sum(1 for pred, target in zip(predictions, targets) if pred == target)

accuracy = correct / total
print(f'Test Accuracy: {accuracy}')

In [18]:
question = "How many active agency customers did we have on January 1st, 2022?"

inputs = tokenizer.encode(question, return_tensors='pt').to(device)

outputs = model.generate(inputs)

sql_query = tokenizer.decode(outputs[0])

print("Generated SQL query:", sql_query)

Generated SQL query: <pad> <unk>How many active agency customers did we have on January 1st, 2022?
